In [81]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil.parser import parse
from tqdm import tqdm

In [2]:
key_fd = open('../keys/gov_data_api_key.txt', mode='r')
govapi_key = key_fd.read(100)
key_fd.close()

In [37]:
# datetime객체에서 문자열로 전처리해줘야 받아옴
start_date = '20201121'
end_date = '20201121'

In [38]:
corona_url = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson"
url = f"{corona_url}?serviceKey={govapi_key}&pageNo=1&numOfRows=1&startCreateDt={start_date}&endCreateDt={end_date}"

In [39]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')

In [40]:
soup

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><confCase>837</confCase><confCaseRate>2.75</confCaseRate><createDt>2020-11-21 14:27:19.271</createDt><criticalRate>0</criticalRate><death>0</death><deathRate>0.00</deathRate><gubun>0-9</gubun><seq>4875</seq><updateDt>null</updateDt></item><item><confCase>1735</confCase><confCaseRate>5.71</confCaseRate><createDt>2020-11-21 14:27:19.271</createDt><criticalRate>0</criticalRate><death>0</death><deathRate>0.00</deathRate><gubun>10-19</gubun><seq>4874</seq><updateDt>null</updateDt></item><item><confCase>5815</confCase><confCaseRate>19.13</confCaseRate><createDt>2020-11-21 14:27:19.271</createDt><criticalRate>0</criticalRate><death>0</death><deathRate>0.00</deathRate><gubun>20-29</gubun><seq>4873</seq><updateDt>null</updateDt></item><item><confCase>3842</confCase><confCaseRate>12.64</confCaseRate><createDt>2020-11-21 14:27:19.271</createDt><crit

In [6]:
# 한 번 요청할 때 들어오는 결과 수
totalCount = soup.select_one('totalCount').text
totalCount

'11'

In [9]:
# 일자
date = soup.select('createDt')
# 구분(성별, 연령별)
gubun = soup.select('gubun')
# 확진자 수
case = soup.select('confCase')
# 확진률
case_rate = soup.select('confCaseRate')
# 사망자 수
death = soup.select('death')
# 사망률
death_rate = soup.select('deathRate')

In [10]:
date_list = [] ; gubun_list = [] ; death_list = [] ; death_rate_list = [] ; case_list = [] ; case_rate_list = []

for i in date:
    date_list.append(parse(i.text).date())
for i in gubun:
    gubun_list.append(i.text)
for i in case:
    case_list.append(int(i.text))
for i in case_rate:
    case_rate_list.append(float(i.text))
for i in death:
    death_list.append(int(i.text))
for i in death_rate:
    death_rate_list.append(float(i.text))

In [11]:
df = pd.DataFrame({
    '일자':date_list,
    '구분':gubun_list, '확진자':case_list,
    '확진률':case_rate_list, '사망자':death_list, '사망률':death_rate_list
})
df

,일자,구분,확진자,확진률,사망자,사망률
0,2020-11-21,0-9,837,2.75,0,0.00
1,2020-11-21,10-19,1735,5.71,0,0.00
2,2020-11-21,20-29,5815,19.13,0,0.00
3,2020-11-21,30-39,3842,12.64,2,0.40
4,2020-11-21,40-49,4158,13.68,4,0.80
5,2020-11-21,50-59,5532,18.20,23,4.57
6,2020-11-21,60-69,4774,15.70,61,12.13
7,2020-11-21,70-79,2403,7.90,160,31.81
8,2020-11-21,80 이상,1307,4.30,253,50.30
9,2020-11-21,여성,16021,52.70,239,47.51


## 시작일 3월 2일

In [78]:
# 오늘 날짜
today = datetime.today()
march_2nd = datetime(2020,3,2)

# 날짜끼리의 사칙연산은 timedelta 클래스로 return됨, where properties such as days, seconds are available.
elapsed_count = (today - march_2nd).days
elapsed_count

294

In [79]:
elapsed_series
elapsed_series = pd.date_range(march_2nd, periods= elapsed_count)

# 원래는 반복문으로 안해도 되는데 정부가 제대로 하는게 없어서

In [80]:
date_list = [] ; gubun_list = [] ; death_list = [] ; death_rate_list = [] ; case_list = [] ; case_rate_list = []

corona_url = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson"
for date in elapsed_series:
    date =  date.strftime('%Y%m%d')

    url = f"{corona_url}?serviceKey={govapi_key}&pageNo=1&numOfRows=1&startCreateDt={date}&endCreateDt={date}"

    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'xml')

    # 일자
    date = soup.select('createDt')
    # 구분(성별, 연령별)
    gubun = soup.select('gubun')
    # 확진자 수
    case = soup.select('confCase')
    # 확진률
    case_rate = soup.select('confCaseRate')
    # 사망자 수
    death = soup.select('death')
    # 사망률
    death_rate = soup.select('deathRate')

    for i in date:
        date_list.append(parse(i.text).date())
    for i in gubun:
        gubun_list.append(i.text)
    for i in case:
        case_list.append(int(i.text))
    for i in case_rate:
        case_rate_list.append(float(i.text))
    for i in death:
        death_list.append(int(i.text))
    for i in death_rate:
        death_rate_list.append(float(i.text))


In [82]:
df = pd.DataFrame({
    '일자':date_list,
    '구분':gubun_list, '확진자':case_list,
    '확진률':case_rate_list, '사망자':death_list, '사망률':death_rate_list
})
df

,일자,구분,확진자,확진률,사망자,사망률
0,2020-04-02,0-9,112,1.14,0,0.00
1,2020-04-02,10-19,515,5.26,0,0.00
2,2020-04-02,20-29,2656,27.14,0,0.00
3,2020-04-02,30-39,1012,10.34,1,0.62
4,2020-04-02,40-49,1312,13.41,1,0.62
...,...,...,...,...,...,...
2812,2020-12-20,60-69,7722,15.55,81,12.02
2813,2020-12-20,70-79,3892,7.84,200,29.67
2814,2020-12-20,80 이상,2374,4.78,357,52.97
2815,2020-12-20,여성,25797,51.94,329,48.81


In [84]:
df.to_csv('./result/성별연령별.csv',sep=',',encoding='utf8')

In [85]:
df = pd.read_csv('./result/성별연령별.csv',sep=',',encoding='utf8')
del df['Unnamed: 0']
df

,일자,구분,확진자,확진률,사망자,사망률
0,2020-04-02,0-9,112,1.14,0,0.00
1,2020-04-02,10-19,515,5.26,0,0.00
2,2020-04-02,20-29,2656,27.14,0,0.00
3,2020-04-02,30-39,1012,10.34,1,0.62
4,2020-04-02,40-49,1312,13.41,1,0.62
...,...,...,...,...,...,...
2812,2020-12-20,60-69,7722,15.55,81,12.02
2813,2020-12-20,70-79,3892,7.84,200,29.67
2814,2020-12-20,80 이상,2374,4.78,357,52.97
2815,2020-12-20,여성,25797,51.94,329,48.81
